In [1]:
import owlready2 as owl
import types
import json
import re
from pprint import pprint as pp
from num2words import num2words

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
ipro = owl.get_ontology("file://ipro.owl").load()
print('#####', ipro)
print(list(ipro.classes()))
print(list(ipro.properties()),'\n')

list_onto = ipro.imported_ontologies[0]
print('#####', list_onto)
print(list(list_onto.classes()))
print(list(list_onto.properties()),'\n')

bag_onto = list_onto.imported_ontologies[0]
print('#####', bag_onto)
print(list(bag_onto.classes()))
print(list(bag_onto.properties()),'\n')

sequence_onto = list_onto.imported_ontologies[1]
print('#####', sequence_onto)
print(list(sequence_onto.classes()))
print(list(sequence_onto.properties()),'\n')

##### get_ontology("http://www.semanticweb.org/ipro#")
[ipro.Album, ipro.Band, ipro.Bass, ipro.MusicalInstrument, ipro.Bassist, ipro.Musician, ipro.Discography, ipro.Drummer, ipro.Genre, ipro.Guitar, ipro.Guitarist, ipro.Keyboard, ipro.Keyboardist, ipro.Organ, ipro.Participation, ipro.Percussions, ipro.Piano, ipro.Singer, ipro.Song, ipro.Synthesizer, ipro.Vocals]
[] 

##### get_ontology("http://www.ontologydesignpatterns.org/cp/owl/list.owl#")
[list.List, list.ListItem]
[list.lastItemOf, list.hasLastItem, list.firstItemOf, list.nextItem, list.previousItem, list.hasFirstItem] 

##### get_ontology("http://www.ontologydesignpatterns.org/cp/owl/bag.owl#")
[bag.Item, bag.Bag]
[bag.size, bag.itemOf, bag.hasItem, bag.itemContent] 

##### get_ontology("http://www.ontologydesignpatterns.org/cp/owl/sequence.owl#")
[owl.Thing]
[sequence.directlyFollows, sequence.directlyPrecedes, sequence.follows, sequence.precedes, rdf-schema.comment, rdf-schema.label, rdf-schema.isDefinedBy, owl.versionInfo, cp

In [3]:
with open('wikidata/artists.json', 'r') as f:
    wikidata_artists = json.load(f)

with open('discogs/artists.json', 'r') as f:
    tmp = json.load(f)
    discogs_artists = {e['discogsID']: e['artist'] for e in tmp}

with open('progarchives/biographies.json', 'r') as f:
    progarchives_bios = json.load(f)

with ipro:
    class wikidataIri(ipro.Band >> str, owl.FunctionalProperty): pass
    class discogsId(ipro.Band >> str, owl.FunctionalProperty): pass
    class progarchivesId(ipro.Band >> str, owl.FunctionalProperty): pass
    class activeSinceYear(ipro.Band >> int, owl.FunctionalProperty): pass
    class imageUrl(ipro.Band >> str): pass
    class discography(ipro.Band >> ipro.Discography, owl.FunctionalProperty): pass
    class discographyOf(ipro.Discography >> ipro.Band, owl.FunctionalProperty):
        inverse_property = ipro.discography

    for artist in wikidata_artists:
        band = ipro.Band(artist['itemLabel']['value'].replace(' ', '_').replace('\'', ''))
        band.wikidataIri = artist['item']['value']
        band.discogsId = artist['discogsId']['value']
        band.progarchivesId = artist['progarchivesId']['value']
        band.comment = [owl.locstr(progarchives_bios[band.progarchivesId], lang = 'en')]
        band.label = [owl.locstr(re.sub(r'\([^\)]*\)', '', discogs_artists[band.discogsId]['name']), lang = 'en')]
        band.activeSinceYear = int(artist['inizio']['value'].split('-')[0])
        band.imageUrl.append(discogs_artists[band.discogsId]['images'][0]['uri'])
        band.discography = ipro.Discography(artist['itemLabel']['value'].replace(' ', '_').replace('\'', '') + '_discography')

print(list(ipro.Band.instances())[:10])

[ipro.Acqua_Fragile, ipro.Alphataurus, ipro.Analogy, ipro.Area, ipro.Arti_e_Mestieri, ipro.Banco_del_Mutuo_Soccorso, ipro.Blocco_Mentale, ipro.Cervello, ipro.De_De_Lind, ipro.Formula_3]


In [4]:
# print all info of Acqua Fragile
print(ipro.Acqua_Fragile)
print(ipro.Acqua_Fragile.comment)
print(ipro.Acqua_Fragile.comment[0].lang)
print(ipro.Acqua_Fragile.label)
print(ipro.Acqua_Fragile.label[0].lang)
print(ipro.Acqua_Fragile.wikidataIri)
print(ipro.Acqua_Fragile.discogsId)
print(ipro.Acqua_Fragile.progarchivesId)
print(ipro.Acqua_Fragile.activeSinceYear)
print(ipro.Acqua_Fragile.imageUrl)
print(ipro.Acqua_Fragile.discography)

ipro.Acqua_Fragile
['Founded in Parma, Italy in 1971 - Disbanded in 1975 - Reformed 2004-2006 and in 2017 as "Acqua Fragile Project".\nThey are perhaps best known for the band that were to supply PFM\'s English singing vocalist Bernado Lanzetti, making his debut with them on Chocolate Kings.\nLanzetti with guitarist Gino Campanini and drummer Piero Canavera had played together in Gli Immortali. Joined on keyboards by Maurizio Mori and bassist Franz Dondi, formerly of I Moschettieri, who released a single in 1967, they shortly changed their name to ACQUA FRAGILE.\nIt was to be two years before their eponymous debut album saw the light of day, due to difficulty in finding a record company that would allow them to release it with English sung lyrics. Musically they bore a resemblance to GENESIS and GENTLE GIANT, with harmony vocals not unlike CROSBY, STILLS, NASH AND YOUNG, no doubt influenced by the time Lanzetti spent in the USA. Lanzetti\'s vocals have a similar feel to Roger Chapman o

In [5]:
with open('discogs/releases.json', 'r') as f:
    tmp = json.load(f)
    discogs_releases = {e['discogsID']: e['releases'] for e in tmp}
    del tmp

with ipro:
    class containsAlbum(ipro.Discography >> ipro.Album, bag_onto.hasItem): pass
    class albumOf(owl.ObjectProperty):
        inverse_property = ipro.containsAlbum
    class previousAlbum(ipro.Album >> ipro.Album, list_onto.previousItem): pass
    class nextAlbum(ipro.Album >> ipro.Album, list_onto.nextItem):
        inverse_property = ipro.previousAlbum
    class releaseYear(ipro.Album >> int, owl.FunctionalProperty): pass

    for band in ipro.Band.instances():
        prev_album = None
        for i, release in enumerate(discogs_releases[band.discogsId]):
            album_title = re.sub(r'[^a-zA-Z_0-9]','',release['title'].replace(' ', '_'))
            album = ipro.Album(album_title + '_album')
            album.label = [owl.locstr(release['title'], lang = 'en')]
            album.comment = [owl.locstr(f'{album.label[0]} is the {num2words(i+1, ordinal=True)} album of {band.label[0]} released in {release["year"]}', lang = 'en')]
            album.releaseYear = release['year']
            band.discography.containsAlbum.append(album)

            if prev_album:
                assert prev_album.releaseYear <= album.releaseYear, "Previous album release year is greater than current album release year"
                prev_album.nextAlbum = album

            prev_album = album


In [6]:
ipro.save('ipro_generated.owl')